In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
connexion = sqlite3.connect('db/vivino.db')
cursor = connexion.cursor()


In [3]:
# 2 - countries Marketing priorities: 

query2a= ("""
SELECT  countries.users_count,
        countries.name, 
        AVG(wines.ratings_average), 
        sum(wines.ratings_count),
        AVG(vintages.price_euros),
        ROUND((SUM(wines.ratings_count)*1.0/countries.users_count), 2) AS ratings_per_user 
        
FROM    wines
        JOIN regions ON wines.region_id = regions.id
        JOIN countries ON regions.country_code = countries.code
        JOIN vintages ON wines.id = vintages.wine_id
GROUP BY countries.name  
ORDER BY countries.users_count DESC, AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC
LIMIT 100
""") 

# for row in cursor.execute(query3):
#     print(row)
cursor.execute(query2a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query2a, connexion)
df



Number of rows: 17
[(12273684, 'États-Unis', 4.506250000000001, 1528232, 300.36616071428574, 0.12), (5973301, 'France', 4.484049930651893, 9634864, 1025.3709986130377, 1.61), (4270717, 'Italie', 4.478492239467868, 11898245, 476.0170214338511, 2.79), (2549989, 'Allemagne', 4.475000000000001, 16609, 282.01666666666665, 0.01), (2264396, 'Espagne', 4.467175572519083, 1132535, 282.1422900763359, 0.5), (1601799, 'Suisse', 4.35, 1047, 123.42500000000001, 0.0), (1123535, 'Portugal', 4.429032258064517, 71717, 161.94903225806445, 0.06), (1022965, 'Australie', 4.5, 142532, 495.00937500000003, 0.14), (629532, 'Argentine', 4.42857142857143, 341337, 111.37749999999998, 0.54), (326757, 'Chili', 4.468, 363988, 129.70519999999996, 1.11), (269649, 'Afrique du Sud', 4.465625000000001, 80449, 87.73968749999997, 0.3), (228185, 'Roumanie', 4.4, 166, 36.08, 0.0), (150549, 'Israël', 4.5, 894, 37.05, 0.01), (102235, 'Hongrie', 4.4642857142857135, 48542, 106.825, 0.47), (95693, 'Grèce', 4.4, 610, 124.0, 0.01), 

,users_count,name,AVG(wines.ratings_average),sum(wines.ratings_count),AVG(vintages.price_euros),ratings_per_user
0,12273684,États-Unis,4.506250,1528232,300.366161,0.12
1,5973301,France,4.484050,9634864,1025.370999,1.61
2,4270717,Italie,4.478492,11898245,476.017021,2.79
3,2549989,Allemagne,4.475000,16609,282.016667,0.01
4,2264396,Espagne,4.467176,1132535,282.142290,0.50
5,1601799,Suisse,4.350000,1047,123.425000,0.00
6,1123535,Portugal,4.429032,71717,161.949032,0.06
7,1022965,Australie,4.500000,142532,495.009375,0.14
8,629532,Argentine,4.428571,341337,111.377500,0.54
9,326757,Chili,4.468000,363988,129.705200,1.11


In [4]:
# 3 - best winery >> The winery Key and name are wrong in the database, join is useless
query3a = ("""
SELECT  wines.winery_id, 
        wineries.name ,
        AVG(wines.ratings_average), 
        AVG(wines.ratings_count)        
        
FROM    wines
        JOIN wineries ON wines.winery_id = wineries.id
GROUP BY wines.winery_id  
ORDER BY AVG(wines.ratings_average) DESC, AVG(wines.ratings_count) DESC
LIMIT 100
""") 

# for row in cursor.execute(query3):
#     print(row)
cursor.execute(query3a)
result = cursor.fetchall()
print("Number of rows:", len(result))
#print (result)
df = pd.read_sql_query(query3a, connexion)
df


Number of rows: 4


,winery_id,name,AVG(wines.ratings_average),AVG(wines.ratings_count)
0,1651,Tenuta Tignanello 'Solaia',4.5,2472.0
1,75712,Corte di Cama Sforzato di Valtellina,4.5,58.0
2,1652,Tignanello,4.4,741.0
3,1471,Siepi,4.3,4981.0


In [5]:
# 3 - best winery 
query3b = ("""
SELECT  wines.name,
        wines.id,
        vintages.name,
        CASE 
                WHEN INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) > 0 THEN
                        SUBSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), 1, INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) - 1) || 
                        SUBSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), INSTR(SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5), wines.name) + LENGTH(wines.name))
                ELSE SUBSTR(vintages.name,1, LENGTH(vintages.name) - 5)
        END AS winery_name,
        wines.winery_id,
        AVG(wines.ratings_average) AS ratings_avg,
        AVG(wines.ratings_count) AS ratings_cnt
        
FROM    wines
        JOIN vintages ON vintages.wine_id = wines.id
       
GROUP BY wines.winery_id  
HAVING ratings_avg >= 4.5 and ratings_cnt> 35000  
ORDER BY AVG(wines.ratings_count)  DESC, AVG(wines.ratings_average) DESC
LIMIT 20
""") 

cursor.execute(query3b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query3b, connexion)
df



Number of rows: 14
[('Sassicaia', 5078, 'Tenuta San Guido Sassicaia 2005', 'Tenuta San Guido ', 1413, 4.599999999999998, 107646.0), ('Brut Champagne', 86684, 'Dom Pérignon Brut Champagne 2000', 'Dom Pérignon ', 212823, 4.625000000000001, 95089.75), ('60 Sessantanni Old Vines Primitivo di Manduria', 11890, 'San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2017', 'San Marzano ', 55648, 4.5, 94289.0), ('Special Selection Cabernet Sauvignon', 66294, 'Caymus Special Selection Cabernet Sauvignon 1997', 'Caymus ', 1301, 4.666666666666667, 80138.66666666667), ('Tignanello', 1652, 'Antinori Tignanello 2005', 'Antinori ', 1252, 4.505, 71223.8), ('Ribera del Duero', 77171, 'Alión Ribera del Duero 2019', 'Alión ', 11049, 4.5, 69278.0), ('Opus One', 1911534, 'Opus One Opus One 2012', ' Opus One', 1368, 4.585714285714286, 68446.0), ('Pauillac (Premier Grand Cru Classé)', 1684223, 'Château Mouton Rothschild Pauillac (Premier Grand Cru Classé) 2009', 'Château Mouton Rothschild ', 64252, 4.55

,name,id,name,winery_name,winery_id,ratings_avg,ratings_cnt
0,Sassicaia,5078,Tenuta San Guido Sassicaia 2005,Tenuta San Guido,1413,4.600000,107646.000000
1,Brut Champagne,86684,Dom Pérignon Brut Champagne 2000,Dom Pérignon,212823,4.625000,95089.750000
2,60 Sessantanni Old Vines Primitivo di Manduria,11890,San Marzano 60 Sessantanni Old Vines Primitivo...,San Marzano,55648,4.500000,94289.000000
3,Special Selection Cabernet Sauvignon,66294,Caymus Special Selection Cabernet Sauvignon 1997,Caymus,1301,4.666667,80138.666667
4,Tignanello,1652,Antinori Tignanello 2005,Antinori,1252,4.505000,71223.800000
5,Ribera del Duero,77171,Alión Ribera del Duero 2019,Alión,11049,4.500000,69278.000000
6,Opus One,1911534,Opus One Opus One 2012,Opus One,1368,4.585714,68446.000000
7,Pauillac (Premier Grand Cru Classé),1684223,Château Mouton Rothschild Pauillac (Premier Gr...,Château Mouton Rothschild,64252,4.550000,62429.500000
8,Unico,77137,Vega Sicilia Unico 2012,Vega Sicilia,11050,4.666667,49002.166667
9,Almaviva,4086,Almaviva Almaviva 2014,Almaviva,17605,4.600000,48075.000000


In [14]:
# 5a - Best 5 wines per type of grapes - most common grapes world wide
query5a = ("""
SELECT  DISTINCT 
        grapes.name,
        most_used_grapes_per_country.grape_id,
        most_used_grapes_per_country.wines_count AS grapes_count
                
FROM    most_used_grapes_per_country
        JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
ORDER BY grapes_count DESC
LIMIT 3

""")
cursor.execute(query5a)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5a, connexion)
df


Number of rows: 3
[('Cabernet Sauvignon', 2, 801751), ('Chardonnay', 5, 604208), ('Pinot Noir', 14, 572334)]


,name,grape_id,grapes_count
0,Cabernet Sauvignon,2,801751
1,Chardonnay,5,604208
2,Pinot Noir,14,572334


In [16]:
# 5b - Best 5 wines per type of grapes -  Country list related to  most common grapes world wide
query5b = ("""

WITH most_common_grapes_WW AS (
        SELECT  DISTINCT 
                grapes.name,
                most_used_grapes_per_country.grape_id,
                most_used_grapes_per_country.wines_count AS grapes_count
               
        FROM    most_used_grapes_per_country
                JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
        ORDER BY grapes_count DESC
        LIMIT 3
)          
SELECT   
        most_used_grapes_per_country.grape_id AS grape_id,
        most_used_grapes_per_country.country_code as country_code
                   
FROM    most_used_grapes_per_country
WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
GROUP BY grape_id, country_code

           
 

""")
cursor.execute(query5b)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5b, connexion)
df

Number of rows: 21
[(2, 'ar'), (2, 'au'), (2, 'cl'), (2, 'es'), (2, 'gr'), (2, 'hr'), (2, 'hu'), (2, 'il'), (2, 'md'), (2, 'ro'), (2, 'us'), (2, 'za'), (5, 'ar'), (5, 'au'), (5, 'fr'), (5, 'it'), (5, 'md'), (5, 'us'), (14, 'ch'), (14, 'fr'), (14, 'us')]


,grape_id,country_code
0,2,ar
1,2,au
2,2,cl
3,2,es
4,2,gr
5,2,hr
6,2,hu
7,2,il
8,2,md
9,2,ro


In [55]:
# 5c - Best 5 wines per type of grapes -  region list  for countries related to most common grapes WW
query5c = ("""

WITH countries_most_common_grapes AS (
    WITH most_common_grapes_WW AS (
            SELECT  DISTINCT 
                    grapes.name,
                    most_used_grapes_per_country.grape_id,
                    most_used_grapes_per_country.wines_count AS grapes_count
                
            FROM    most_used_grapes_per_country
                    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
            ORDER BY grapes_count DESC
            LIMIT 3
    )          
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
    GROUP BY grape_id, country_code
)
           
SELECT
    regions.name AS region_name,
    regions.id AS region_id,
    regions.country_code AS region_country_code,
    countries_most_common_grapes.country_code AS prev_country_code,
    countries_most_common_grapes.grape_id AS grape_id
    
    
FROM    
    regions
    JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
    
ORDER BY grape_id, region_id, region_country_code

           
        
""")


cursor.execute(query5c)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5c, connexion)
df

Number of rows: 410
[('South Australia', 7, 'au', 'au', 2), ('California', 24, 'us', 'us', 2), ('Napa Valley', 25, 'us', 'us', 2), ('Sonoma County', 26, 'us', 'us', 2), ('Oakville', 42, 'us', 'us', 2), ('Willamette Valley', 51, 'us', 'us', 2), ('Yamhill County', 55, 'us', 'us', 2), ('Lodi', 65, 'us', 'us', 2), ('Howell Mountain', 85, 'us', 'us', 2), ('Santa Cruz Mountains', 86, 'us', 'us', 2), ('Stags Leap District', 88, 'us', 'us', 2), ('Calistoga', 91, 'us', 'us', 2), ('Russian River Valley', 94, 'us', 'us', 2), ('Los Carneros', 96, 'us', 'us', 2), ('Sta. Rita Hills', 99, 'us', 'us', 2), ('Alexander Valley', 100, 'us', 'us', 2), ('Rutherford', 105, 'us', 'us', 2), ('Walla Walla Valley', 324, 'us', 'us', 2), ('Columbia Valley', 327, 'us', 'us', 2), ('Barossa Valley', 397, 'au', 'au', 2), ('Ribera del Duero', 405, 'es', 'es', 2), ('Cachapoal Valley', 411, 'cl', 'cl', 2), ('Margaret River', 425, 'au', 'au', 2), ('Toro', 430, 'es', 'es', 2), ('Clare Valley', 444, 'au', 'au', 2), ('Mendoz

,region_name,region_id,region_country_code,prev_country_code,grape_id
0,South Australia,7,au,au,2
1,California,24,us,us,2
2,Napa Valley,25,us,us,2
3,Sonoma County,26,us,us,2
4,Oakville,42,us,us,2
...,...,...,...,...,...
405,Champagne Grand Cru 'Ambonnay',4624,fr,fr,14
406,Champagne Grand Cru 'Avize',4625,fr,fr,14
407,Champagne Grand Cru 'Le Mesnil-sur-Oger',4631,fr,fr,14
408,Champagne Grand Cru,4638,fr,fr,14


In [4]:
# 5c - Best 5 wines per type of grapes - all together and extracting the 5 best for each grapes from all region in  the region list
query5d = ("""

WITH countries_most_common_grapes AS (
    WITH most_common_grapes_WW AS (
            SELECT  DISTINCT 
                    grapes.name,
                    most_used_grapes_per_country.grape_id,
                    most_used_grapes_per_country.wines_count AS grapes_count
                
            FROM    most_used_grapes_per_country
                    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
            ORDER BY grapes_count DESC
            LIMIT 3
    )          
    SELECT   
            most_used_grapes_per_country.grape_id AS grape_id,
            most_used_grapes_per_country.country_code as country_code
                    
    FROM    most_used_grapes_per_country
    WHERE   grape_id IN (SELECT grape_id FROM most_common_grapes_WW)
    GROUP BY grape_id, country_code
),

ranked_wines AS ( 
        SELECT
                countries_most_common_grapes.grape_id AS grape_id,
                ROW_NUMBER() OVER(
                        PARTITION BY countries_most_common_grapes.grape_id 
                        ORDER BY wines.ratings_average DESC, 
                        wines.ratings_count DESC
                        ) as rn,
                wines.name as wines_name,
                wines.ratings_average AS ratings,
                wines.ratings_count as ratings_count,
                regions.name AS region_name,
                regions.country_code AS region_country_code
                --countries_most_common_grapes.country_code AS prev_country_code
                --wines.region_id as wines_region_id,
                --regions.id AS region_id,
                
                   
                
        FROM    regions
                JOIN countries_most_common_grapes ON regions.country_code = countries_most_common_grapes.country_code
                JOIN wines ON regions.id = wines.region_id
)
           
SELECT * FROM ranked_wines WHERE rn <= 5
        
""")


cursor.execute(query5d)
result = cursor.fetchall()
print("Number of rows:", len(result))
print (result)
df = pd.read_sql_query(query5d, connexion)
df

Number of rows: 15
[(2, 1, 'Cabernet Sauvignon', 4.8, 2941, 'Rutherford', 'us'), (2, 2, 'Mágico', 4.8, 146, 'Rioja', 'es'), (2, 3, 'Unico', 4.7, 45140, 'Ribera del Duero', 'es'), (2, 4, 'Special Selection Cabernet Sauvignon', 4.7, 41236, 'Napa Valley', 'us'), (2, 5, 'Grange', 4.7, 24356, 'South Australia', 'au'), (5, 1, 'Cabernet Sauvignon', 4.8, 2941, 'Rutherford', 'us'), (5, 2, 'Amarone della Valpolicella Classico Riserva', 4.8, 587, 'Amarone della Valpolicella Classico', 'it'), (5, 3, 'Fratini Bolgheri Superiore', 4.8, 153, 'Bolgheri Superiore', 'it'), (5, 4, 'Cristal Rosé Vinothèque', 4.8, 88, 'Champagne', 'fr'), (5, 5, 'Batard-Montrachet Grand Cru', 4.8, 34, 'Bâtard-Montrachet Grand Cru', 'fr'), (14, 1, 'Cabernet Sauvignon', 4.8, 2941, 'Rutherford', 'us'), (14, 2, 'Cristal Rosé Vinothèque', 4.8, 88, 'Champagne', 'fr'), (14, 3, 'Batard-Montrachet Grand Cru', 4.8, 34, 'Bâtard-Montrachet Grand Cru', 'fr'), (14, 4, 'Sauternes', 4.7, 44126, 'Sauternes', 'fr'), (14, 5, 'Special Selectio

,grape_id,rn,wines_name,ratings,ratings_count,region_name,region_country_code
0,2,1,Cabernet Sauvignon,4.8,2941,Rutherford,us
1,2,2,Mágico,4.8,146,Rioja,es
2,2,3,Unico,4.7,45140,Ribera del Duero,es
3,2,4,Special Selection Cabernet Sauvignon,4.7,41236,Napa Valley,us
4,2,5,Grange,4.7,24356,South Australia,au
5,5,1,Cabernet Sauvignon,4.8,2941,Rutherford,us
6,5,2,Amarone della Valpolicella Classico Riserva,4.8,587,Amarone della Valpolicella Classico,it
7,5,3,Fratini Bolgheri Superiore,4.8,153,Bolgheri Superiore,it
8,5,4,Cristal Rosé Vinothèque,4.8,88,Champagne,fr
9,5,5,Batard-Montrachet Grand Cru,4.8,34,Bâtard-Montrachet Grand Cru,fr
